In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
import time
from openpyxl.styles import Font, Color, PatternFill, Border, Side, Alignment
#from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from splinter import Browser

def click_on_something(strings,types_,first = False,by_text = '',used_already = []):
    found = False
    my_dict = {}
    my_dict["strings"] = strings
    my_dict["types"] = types_
    #print(my_dict)
    j = 0
    while found == False:
        try: 
            to_click = browser
            for i in range(0,len(my_dict["strings"])):
                if my_dict["types"][i] == "id":
                    to_click = to_click.find_by_id(my_dict["strings"][i])
                elif my_dict["types"][i] == "css":
                    blah = f".{my_dict['strings'][i]}"
                    to_click = to_click.find_by_css(blah)
                    #In the case of selecting boundlists, must select the one that hasent been used yet
                    if used_already and my_dict['strings'][i] == 'x-boundlist':
                        for obj in to_click:
                            if obj not in used_already:
                                to_click = obj
                else:
                    print("types_ invalid input: must be id or css")
                #print(f"to click: {to_click}")
            #If not looking to click on the first returned object
            if first == False:
                #if searching element by text (e.g an option in a drop down)
                if by_text:
                    clicked = False
                    for obj in to_click:
                        if obj.text == by_text:
                            obj.click()
                            clicked = True
                    if clicked == False:
                        print("by_text invalid input: text not found")
                        break
                #if not searching by text (e.g a drop down)
                else:
                    to_click.click()
                found = True
            #if looking to click on the first returned object
            elif first == True:
                to_click.first.click()
                found = True
            else:
                print("first invalid input: must be True or False")
                break
            #return to_click
        
        except AttributeError:
            j += 1
            print(f"looking for element. attempt number {j}")
            
        if j > 10:
            print("Either app didn't load properly, or internet is down/weak")
            break

def scrape(num_,browser,app_dict):
    browser.visit(f'{base_url}{app_dict["url"][num_]}')

    loaded = False
    while loaded == False:
        click_on_something(["tab-1013"],["id"])
        try:
            click_on_something(["cmp_series_SERIE-triggerWrap","x-trigger-index-0"],["id","css"])
            loaded = True
        except:
            click_on_something(["tab-1012"],["id"])
            print("walao")

    #Clicking the todas las series button
    #select_all = browser.find_by_css(".list-ct").find_by_css(".x-column-header-text").first
    #select_all.click()
    series_clicked = False
    while series_clicked == False:
        click_on_something(["list-ct","x-column-header-text"],["css","css"],True)
        test = browser.find_by_id("cmp_series_SERIE-inputEl")._element.get_attribute('value')
        if test:
            series_clicked = True

    #Clicking on cifras drop down
    #cifras_drop = browser.find_by_id("cmp_cuadro_PRESENTACION").find_by_css(".x-trigger-index-0")
    #cifras_drop.click()
    cifras_drop_clicked = False
    while cifras_drop_clicked == False:
        try:
            cifras_drop = click_on_something(["cmp_cuadro_PRESENTACION","x-trigger-index-0"],["id","css"])
            cifras_drop_clicked = True
        except:
            print("not found yet...")



    #Clicking on millones de pesos
    cifras_opts = click_on_something(["x-boundlist","x-boundlist-item"],["css","css"],False,"Millones de pesos")

    #Upon clicking the dropdown, an x-boundlist is generated, must add it to used_aleady to we dont use it again
    used_already = []
    used_already = add_to_used(used_already,browser.find_by_css(".x-boundlist"))
    used_already

    #Clicking on year lower limit drop down
    #low_lim_drop = browser.find_by_id("cmp_cuadro_DE").find_by_css(".x-trigger-cell").find_by_css(".x-trigger-index-0")
    #low_lim_drop.click()
    low_lim_clicked = False
    while low_lim_clicked == False:
        try:
            low_lim_drop = click_on_something(["cmp_cuadro_DE","x-trigger-cell","x-trigger-index-0"],["id","css","css"])
            low_lim_clicked = True
        except:
            print("not found yet...")

    #Clicking on the latest year in the low_lim_opts option menu
    low_lim_opts = click_on_something(["x-boundlist","x-boundlist-item"],["css","css"],True,'',used_already)
    #adding bound list generated by clicking low_lim_drop to used_already
    used_already = add_to_used(used_already,browser.find_by_css(".x-boundlist"))
    used_already

    #Clicking on year upper limit drop down
    #up_lim_drop = browser.find_by_id("cmp_cuadro_A").find_by_css(".x-trigger-index-0")
    #up_lim_drop.click()
    up_lim_clicked = False
    while up_lim_clicked == False:
        try:
            up_lim_drop = click_on_something(["cmp_cuadro_A","x-trigger-index-0"],["id","css"])
            up_lim_clicked = True
        except:
            print("not found yet...")
    #Clicking on the latest year in the up_lim_opts option menu
    low_lim_opts = click_on_something(["x-boundlist","x-boundlist-item"],["css","css"],True,'',used_already)
    #adding bound list generated by clicking low_lim_drop to used_already
    used_already = add_to_used(used_already,browser.find_by_css(".x-boundlist"))
    used_already

    #Consultar Series (clicking the button and suppressing warnings until the page loads the data)
    #consultar_series = browser.find_by_id("cmp_series_CONSULTAR_SERIES-btnEl")
    #consultar_series.click()
    success = False
    while success == False:
        clicked = False
        try:
            consultar_series = click_on_something(["cmp_series_CONSULTAR_SERIES-btnEl"],["id"])
            clicked = True
            print("consultar button clicked")
        except:
            print("no consultar button")
        if clicked == True:
            try:
                alert = browser.get_alert()
                alert.accept()
                print("alert accepted")
            except:
                print("there is no alert")
                time.sleep(0.5)
                test = browser.find_by_css(".x-mask-msg")
                #Sometimes the button isnt clicked, thus...
                print(f'test = {test}')
                if test == []:
                    consultar_series = click_on_something(["cmp_series_CONSULTAR_SERIES-btnEl"],["id"])
                    print("had to click again")
                success = True

    loaded = False

    while loaded == False:
        try:
            the_table = browser.find_by_id("GridSeries")
            header_row = the_table.find_by_css(".x-box-inner")[1]
            loaded = True
        except:
            print("loading...")
            #sometimes the consultar button isnt clicked in the previous step, thus...
                
            
    #Finding Headers of the table
    header_divs = header_row.find_by_css(".x-unselectable")
    idx = 0
    headers = []
    for header in header_divs:
        if idx == 0:
            headers.append(header.find_by_tag("span").find_by_tag("label")._element.get_attribute("innerHTML").strip())
            idx += 1
        else:
            headers.append(header.find_by_tag("span")._element.get_attribute("innerHTML").replace("\n","").replace("\r","").strip())

    #Scraping the data and creating a data frame
    table_large = the_table.find_by_id("GridSeries-body")

    table = table_large.find_by_tag("table")

    rows = table.find_by_tag("tr")

    data_rows = list(rows)[1:]

    the_dict = dict((el,[]) for el in headers)

    for row in data_rows:
        to_append = row.text.split("\n")
        for k in range(0,len(the_dict)):
            the_dict[headers[k]].append(to_append[k]) 

    df =pd.DataFrame(the_dict)
    return df

def store(df_dict):
    #Loading the excel file
    wb = load_workbook(filename='N:/Lsolis/eco/SHCP/Historicos/test.xlsm', read_only=False, keep_vba=True)
    idx = 0
    for worksheet in df_dict["sheet"]:
        #Loading the sheet
        sheet_ranges = wb[worksheet]
        
        #determining whether or not a new row(s) must be inserted
        done = False
        row_ = 3
        while done == False:
            if sheet_ranges[f'A{row_}'].value or sheet_ranges[f'A{row_}'].value == 0:
                #print(sheet_ranges[f'A{row_}'].value)
                row_ += 1
            else:
                done = True

        last_row = row_ - 1

        current_year = int(sheet_ranges[f'A{last_row}'].value[-4:])

        current_month = int(sheet_ranges[f'A{last_row}'].value[:2])

        if current_month == 12:
            current_year += 1

        previous_year = current_year - 1

        done = False
        row_ = 4
        while done == False:
            if sheet_ranges[f'A{row_}'].value or sheet_ranges[f'A{row_}'].value == 0:
                if int(sheet_ranges[f'A{row_}'].value[-4:]) == previous_year:
                    last_row_previous = row_
                    #print(sheet_ranges[f'A{row_}'].value)  
                row_ += 1
            else:
                done = True

        if len(df_dict["df"][idx]) > (last_row-last_row_previous):
            sheet_ranges.insert_rows(last_row + 1, len(df_dict["df"][idx])-(last_row - last_row_previous))
        
        #finding row range to which values will be inserted
        ins_begin = last_row_previous + 1
        ins_end = ins_begin + len(df_dict["df"][idx]) - 1
        
        
        #Making sure headers in excel are equal to headers in dataframe
        sheet_col_heads = []
        sheet_col_heads2 = []
        for col in range(0,count_columns(sheet_ranges)):
            sheet_col_heads.append(sheet_ranges['A3'].offset(0,col).value.replace("_x000D_",""))
            sheet_col_heads2.append(sheet_ranges['A3'].offset(0,col).value)
              
        stripped_col_heads = []
        for head in sheet_col_heads:
            stripped_col_heads.append(head.strip())
            
        stripped_col_heads2 = []
        for head in sheet_col_heads2:
            stripped_col_heads2.append(head.strip())

        compare = list(df_dict["df"][idx].columns)
        compare2 = []
        
        for header in compare:
            compare2.append(header.replace("\r","").replace("\n","").strip())
            
        print(f"{worksheet}: {stripped_col_heads == compare2}")
        
        
        if stripped_col_heads == compare2:
            pass
        else:
            print(f"sheet column headers don't agree with scraped scraped column headers")
            break
            
            
        #Insert Values from dataframe into excel with some format
        row_height = sheet_ranges.row_dimensions[last_row_previous].height
        for col in range(1,len(stripped_col_heads) + 1):
            for rw in range(ins_begin, ins_end + 1):
                #pasting value in cell
                try:
                    sheet_ranges.cell(rw,col).value = float((df_dict["df"][idx][stripped_col_heads[col - 1]][rw-ins_begin]).replace(",",""))
                    sheet_ranges.cell(rw,col).number_format = '#,###.0'
                except ValueError:
                    sheet_ranges.cell(rw,col).value = (df_dict["df"][idx][stripped_col_heads[col - 1]][rw-ins_begin]).replace(",","")
                #setting font style and size    
                sheet_ranges.cell(rw,col).font = Font(name='Arial', size=9)
                #setting border of cell
                border = Border(left=Side(border_style='thin', color='E4E4E4'),
                                right=Side(border_style='thin', color='E4E4E4'),
                                top=Side(border_style='thin', color='E4E4E4'),
                                bottom=Side(border_style='thin', color='E4E4E4'))
                sheet_ranges.cell(rw,col).border = border
                #setting alignment of text in cell
                alignment=Alignment(horizontal='right',
                                    vertical='top')
                sheet_ranges.cell(rw,col).alignment = alignment
                #setting row height
                sheet_ranges.row_dimensions[rw].height = row_height
                #setting fill color of cell
                greyFill = PatternFill(start_color='DCDCDC',
                           end_color='DCDCDC',
                           fill_type='solid')
                whiteFill = PatternFill(start_color='FFFFFF',
                    end_color='FFFFFF',
                    fill_type='solid')
                if rw % 2 == 0:
                    sheet_ranges.cell(rw,col).fill = greyFill 
                else:
                    sheet_ranges.cell(rw,col).fill = whiteFill
            print("column inserted successfully")
        
        idx += 1
    wb.save(filename = 'N:/Lsolis/eco/SHCP/Historicos/test.xlsm')

def add_to_used(used_already,xboundlists):
    for lst in xboundlists:
        el_id = lst._element.get_attribute('id')
        if el_id not in used_already:
            used_already.append(el_id)
    return used_already

def count_columns(sheet_ranges):
    non_empty = True
    col_ = 1
    col_count = 0
    while non_empty:
        if sheet_ranges['A3'].offset(0,col_ - 1).value:
            col_ +=1
            col_count += 1
        else:
            non_empty = False
    return col_count

app_dict = {
    "title":[
        "Balance del Sector Púbilco",
        "Situación Financiera del Sector Público",
        "Ahorro Corriente del Sector Público Presupuestario",
        "Financiamiento de la Inversión Física y Financiera Presupuestaria",
        "Financiamiento del Balance Público por Institución",
        "Financiamiento del Balance Público por Sector",
        "Ingresos Presupuestarios del Sector Público",
        "Ingresos Petroleros del Sector Público",
        "Ingresos Presupuestarios del Gobierno Federal, Petroleros y No petroleros",
        "Ingresos Presupuestarios del Gobierno Federal (Resumen Artículo 1 LIF)",
        "Ingresos Propios de Organismos y Empresas Bajo Control Presupuestario Directo",
        "Gasto Neto del Sector Público Presupuestario",
        "Gasto Programable del Sector Público Presupuestario",
        "Gasto Programable del Sector Público Presupuestario. Clasificación Funcional",
        "Gasto Programable del Sector Público Presupuestario. Clasidicación Funcional-Económica",
        "Servicios Personales del Sector Público Presupuestario. Clasfificación Funcional",
        "Otros Gastos de Operación del Sector Público Presupuestario. Clasificación Funcional",
        "Subsidios, Transferencias y Aportaciones del Sector Público Presupuestario distintos de Servicios Personales. Clasificación Funcional",
        "Ayudas y Otris Gastos Corrientes del Sector Oúblico Presupuestario. Clasificación Funcional",
        "Invesrsión Física del Sector Público Presupuestario. Clasificación Funcional",
        "Subsidios, Transferencias y Aportaciones del Sector Público Presupuestario Incluyendo Servicios Personales. Clasificación Funcional",
        "Inversión física presupuestaria",
        "Costo Financiero del Sector Público Presupuestario",
        "Costo Financiero del Sector Público Presupuestario. Clasificación Institucional",
        "Gasto del Gobierno Federal en Clasificación Económica",
        "Principales Beneficiarios de Transferencias del Gobierno Federal. Clasificación Funcional",
        "Participaciones a Entidades Federativas y Municipios por Fondo",
        "Adeudos de Ejercicios Fiscales Anteriores (ADEFAS). Clasificación Económica",
        "Adeudos de Ejercicios Fiscales Anteriores (ADEFAS). Clasificación Administrativa"
    ],
    "url":[
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Balance%20del%20Sector%20P%C3%BAblico&param_formato=1&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Situaci%C3%B3n%20Financiera%20del%20Sector%20P%C3%BAblico&param_formato=2&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ahorro%20Corriente%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=18&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Financiamiento%20de%20la%20Inversi%C3%B3n%20F%C3%ADsica%20y%20Financiera%20Presupuestaria&param_formato=19&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Financiamiento%20del%20Balance%20P%C3%BAblico%20por%20Instituci%C3%B3n&param_formato=20&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Financiamiento%20del%20Balance%20P%C3%BAblico%20por%20Sector&param_formato=21&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Presupuestarios%20del%20Sector%20P%C3%BAblico&param_formato=3&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Petroleros%20del%20Sector%20P%C3%BAblico&param_formato=29&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Presupuestarios%20del%20Gobierno%20Federal%2C%20Petroleros%20y%20No%20petroleros&param_formato=2&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Presupuestarios%20del%20Gobierno%20Federal%20%20(Resumen%20Art%C3%ADculo%201%20LIF)&param_formato=3&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Propios%20de%20Organismos%20y%20Empresas%20Bajo%20Control%20Presupuestario%20Directo&param_formato=4&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=3&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Neto%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=5&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20programable%20del%20sector%20p%C3%BAblico%20presupuestario&param_formato=15&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Programable%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=8&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Programable%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional-Econ%C3%B3mica&param_formato=9&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Servicios%20Personales%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=23&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Otros%20Gastos%20de%20Operaci%C3%B3n%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=24&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Subsidios%2C%20Transferencias%20y%20Aportaciones%20del%20Sector%20P%C3%BAblico%20Presupuestario%20distintos%20de%20Servicios%20Personales.%20Clasificaci%C3%B3n%20Funcional&param_formato=25&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ayudas%20y%20Otros%20Gastos%20Corrientes%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=26&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Inversi%C3%B3n%20F%C3%ADsica%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=27&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Subsidios%2C%20Transferencias%20y%20Aportaciones%20del%20Sector%20P%C3%BAblico%20Presupuestario%20Incluyendo%20Servicios%20Personales.%20Clasificaci%C3%B3n%20Funcional&param_formato=28&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Inversi%C3%B3n%20f%C3%ADsica%20presupuestaria&param_formato=11&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Costo%20Financiero%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=12&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Costo%20Financiero%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Institucional&param_formato=13&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20del%20Gobierno%20Federal%20en%20Clasificaci%C3%B3n%20Econ%C3%B3mica&param_formato=29&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Principales%20Beneficiarios%20de%20Transferencias%20del%20Gobierno%20Federal.%20Clasificaci%C3%B3n%20Funcional&param_formato=10&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Participaciones%20a%20Entidades%20Federativas%20y%20Municipios%20por%20Fondo&param_formato=17&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Adeudos%20de%20Ejercicios%20Fiscales%20Anteriores%20(ADEFAS).%20Clasificaci%C3%B3n%20Econ%C3%B3mica&param_formato=14&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Adeudos%20de%20Ejercicios%20Fiscales%20Anteriores%20(ADEFAS).%20Clasificaci%C3%B3n%20Administrativa&param_formato=15&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&"
    ],
    "sheet":[
        "1.1.1S",
        "1.1.2S",
        "1.1.3S",
        "1.1.4S",
        "1.1.5S",
        "1.1.6S",
        "1.2.1S",
        "1.2.2S",
        "1.2.3S",
        "1.2.4S",
        "1.2.5S",
        "1.3.1S",
        "1.3.2S",
        "1.3.3S",
        "1.3.4S",
        "1.3.5S",
        "1.3.6S",
        "1.3.7S",
        "1.3.8S",
        "1.3.9S",
        "1.3.10S",
        "1.3.11S",
        "1.3.12S",
        "1.3.13S",
        "1.3.14S",
        "1.3.15S",
        "1.3.16S",
        "1.3.17S",
        "1.3.18S"
    ]
}

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

base_url = "http://presto.hacienda.gob.mx"
#table_url = "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Balance%20del%20Sector%20P%C3%BAblico&param_formato=1&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1& "

#url = "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Balance%20del%20Sector%20P%C3%BAblico&param_formato=1&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&"
#url = "http://www.banxico.org.mx/"


df_dict = {
    "title":[],
    "df":[],
    "sheet":[]
}
for i in range(0,len(app_dict["title"])):
    df = scrape(i,browser,app_dict)
    title = browser.find_by_css(".cls-title-table").find_by_css(".cls-title-main").text
    df_dict["title"].append(title)
    df_dict["df"].append(df)
    df_dict["sheet"].append(app_dict["sheet"][i])

store(df_dict)

looking for element. attempt number 1
looking for element. attempt number 1
walao
walao
walao
walao
walao
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
consultar button clicked
alert accepted
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar b

loading...
walao
walao
walao
walao
walao
walao
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
consultar button clicked
there is no alert
test = [<splinter.driver.webdriver.WebDriverElement object at 0x0D492B10>, <splinter.driver.webdriver.WebDriverElement object at 0x0D494C50>]
loading...
walao
walao
walao
walao
walao
walao
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
consultar button clicked
there is no alert
test = [<splinter.driver.webdriver.WebDriverElement object at 0x0D495570>, <splinter.driver.webdriver.WebDriverElement object at 0x0D495F90>]
loading...
walao
walao
walao
walao
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not foun

column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
1.2.3S: True
c

column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
1.3.6S: True
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
c

1.3.18S: True
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully


In [50]:
def click_on_something(strings,types_,first = False,by_text = '',used_already = []):
    found = False
    my_dict = {}
    my_dict["strings"] = strings
    my_dict["types"] = types_
    #print(my_dict)
    j = 0
    while found == False:
        try: 
            to_click = browser
            for i in range(0,len(my_dict["strings"])):
                if my_dict["types"][i] == "id":
                    to_click = to_click.find_by_id(my_dict["strings"][i])
                elif my_dict["types"][i] == "css":
                    blah = f".{my_dict['strings'][i]}"
                    to_click = to_click.find_by_css(blah)
                    #In the case of selecting boundlists, must select the one that hasent been used yet
                    if used_already and my_dict['strings'][i] == 'x-boundlist':
                        for obj in to_click:
                            if obj not in used_already:
                                to_click = obj
                else:
                    print("types_ invalid input: must be id or css")
                #print(f"to click: {to_click}")
            #If not looking to click on the first returned object
            if first == False:
                #if searching element by text (e.g an option in a drop down)
                if by_text:
                    clicked = False
                    for obj in to_click:
                        if obj.text == by_text:
                            obj.click()
                            clicked = True
                    if clicked == False:
                        print("by_text invalid input: text not found")
                        break
                #if not searching by text (e.g a drop down)
                else:
                    to_click.click()
                found = True
            #if looking to click on the first returned object
            elif first == True:
                to_click.first.click()
                found = True
            else:
                print("first invalid input: must be True or False")
                break
            #return to_click
        
        except AttributeError:
            j += 1
            print(f"looking for element. attempt number {j}")
            
        if j > 10:
            print("Either app didn't load properly, or internet is down/weak")
            break

In [51]:
def scrape(num_,browser,app_dict):
    browser.visit(f'{base_url}{app_dict["url"][num_]}')

    loaded = False
    while loaded == False:
        click_on_something(["tab-1013"],["id"])
        try:
            click_on_something(["cmp_series_SERIE-triggerWrap","x-trigger-index-0"],["id","css"])
            loaded = True
        except:
            click_on_something(["tab-1012"],["id"])
            print("walao")

    #Clicking the todas las series button
    #select_all = browser.find_by_css(".list-ct").find_by_css(".x-column-header-text").first
    #select_all.click()
    series_clicked = False
    while series_clicked == False:
        click_on_something(["list-ct","x-column-header-text"],["css","css"],True)
        test = browser.find_by_id("cmp_series_SERIE-inputEl")._element.get_attribute('value')
        if test:
            series_clicked = True

    #Clicking on cifras drop down
    #cifras_drop = browser.find_by_id("cmp_cuadro_PRESENTACION").find_by_css(".x-trigger-index-0")
    #cifras_drop.click()
    cifras_drop_clicked = False
    while cifras_drop_clicked == False:
        try:
            cifras_drop = click_on_something(["cmp_cuadro_PRESENTACION","x-trigger-index-0"],["id","css"])
            cifras_drop_clicked = True
        except:
            print("not found yet...")



    #Clicking on millones de pesos
    cifras_opts = click_on_something(["x-boundlist","x-boundlist-item"],["css","css"],False,"Millones de pesos")

    #Upon clicking the dropdown, an x-boundlist is generated, must add it to used_aleady to we dont use it again
    used_already = []
    used_already = add_to_used(used_already,browser.find_by_css(".x-boundlist"))
    used_already

    #Clicking on year lower limit drop down
    #low_lim_drop = browser.find_by_id("cmp_cuadro_DE").find_by_css(".x-trigger-cell").find_by_css(".x-trigger-index-0")
    #low_lim_drop.click()
    low_lim_clicked = False
    while low_lim_clicked == False:
        try:
            low_lim_drop = click_on_something(["cmp_cuadro_DE","x-trigger-cell","x-trigger-index-0"],["id","css","css"])
            low_lim_clicked = True
        except:
            print("not found yet...")

    #Clicking on the latest year in the low_lim_opts option menu
    low_lim_opts = click_on_something(["x-boundlist","x-boundlist-item"],["css","css"],True,'',used_already)
    #adding bound list generated by clicking low_lim_drop to used_already
    used_already = add_to_used(used_already,browser.find_by_css(".x-boundlist"))
    used_already

    #Clicking on year upper limit drop down
    #up_lim_drop = browser.find_by_id("cmp_cuadro_A").find_by_css(".x-trigger-index-0")
    #up_lim_drop.click()
    up_lim_clicked = False
    while up_lim_clicked == False:
        try:
            up_lim_drop = click_on_something(["cmp_cuadro_A","x-trigger-index-0"],["id","css"])
            up_lim_clicked = True
        except:
            print("not found yet...")
    #Clicking on the latest year in the up_lim_opts option menu
    low_lim_opts = click_on_something(["x-boundlist","x-boundlist-item"],["css","css"],True,'',used_already)
    #adding bound list generated by clicking low_lim_drop to used_already
    used_already = add_to_used(used_already,browser.find_by_css(".x-boundlist"))
    used_already

    #Consultar Series (clicking the button and suppressing warnings until the page loads the data)
    #consultar_series = browser.find_by_id("cmp_series_CONSULTAR_SERIES-btnEl")
    #consultar_series.click()
    success = False
    while success == False:
        clicked = False
        try:
            consultar_series = click_on_something(["cmp_series_CONSULTAR_SERIES-btnEl"],["id"])
            clicked = True
            print("consultar button clicked")
        except:
            print("no consultar button")
        if clicked == True:
            try:
                alert = browser.get_alert()
                alert.accept()
                print("alert accepted")
            except:
                print("there is no alert")
                time.sleep(0.5)
                test = browser.find_by_css(".x-mask-msg")
                #Sometimes the button isnt clicked, thus...
                print(f'test = {test}')
                if test == []:
                    consultar_series = click_on_something(["cmp_series_CONSULTAR_SERIES-btnEl"],["id"])
                    print("had to click again")
                success = True

    loaded = False

    while loaded == False:
        try:
            the_table = browser.find_by_id("GridSeries")
            header_row = the_table.find_by_css(".x-box-inner")[1]
            loaded = True
        except:
            print("loading...")
            #sometimes the consultar button isnt clicked in the previous step, thus...
                
            
    #Finding Headers of the table
    header_divs = header_row.find_by_css(".x-unselectable")
    idx = 0
    headers = []
    for header in header_divs:
        if idx == 0:
            headers.append(header.find_by_tag("span").find_by_tag("label")._element.get_attribute("innerHTML").strip())
            idx += 1
        else:
            headers.append(header.find_by_tag("span")._element.get_attribute("innerHTML").replace("\n","").replace("\r","").strip())

    #Scraping the data and creating a data frame
    table_large = the_table.find_by_id("GridSeries-body")

    table = table_large.find_by_tag("table")

    rows = table.find_by_tag("tr")

    data_rows = list(rows)[1:]

    the_dict = dict((el,[]) for el in headers)

    for row in data_rows:
        to_append = row.text.split("\n")
        for k in range(0,len(the_dict)):
            the_dict[headers[k]].append(to_append[k]) 

    df =pd.DataFrame(the_dict)
    return df

In [71]:
def store(df_dict):
    #Loading the excel file
    wb = load_workbook(filename='N:/Lsolis/eco/SHCP/Historicos/test.xlsm', read_only=False, keep_vba=True)
    idx = 0
    for worksheet in df_dict["sheet"]:
        #Loading the sheet
        sheet_ranges = wb[worksheet]
        
        #determining whether or not a new row(s) must be inserted
        done = False
        row_ = 3
        while done == False:
            if sheet_ranges[f'A{row_}'].value or sheet_ranges[f'A{row_}'].value == 0:
                #print(sheet_ranges[f'A{row_}'].value)
                row_ += 1
            else:
                done = True

        last_row = row_ - 1

        current_year = int(sheet_ranges[f'A{last_row}'].value[-4:])

        current_month = int(sheet_ranges[f'A{last_row}'].value[:2])

        if current_month == 12:
            current_year += 1

        previous_year = current_year - 1

        done = False
        row_ = 4
        while done == False:
            if sheet_ranges[f'A{row_}'].value or sheet_ranges[f'A{row_}'].value == 0:
                if int(sheet_ranges[f'A{row_}'].value[-4:]) == previous_year:
                    last_row_previous = row_
                    #print(sheet_ranges[f'A{row_}'].value)  
                row_ += 1
            else:
                done = True

        if len(df_dict["df"][idx]) > (last_row-last_row_previous):
            sheet_ranges.insert_rows(last_row + 1, len(df_dict["df"][idx])-(last_row - last_row_previous))
        
        #finding row range to which values will be inserted
        ins_begin = last_row_previous + 1
        ins_end = ins_begin + len(df_dict["df"][idx]) - 1
        
        
        #Making sure headers in excel are equal to headers in dataframe
        sheet_col_heads = []
        sheet_col_heads2 = []
        for col in range(0,count_columns(sheet_ranges)):
            sheet_col_heads.append(sheet_ranges['A3'].offset(0,col).value.replace("_x000D_",""))
            sheet_col_heads2.append(sheet_ranges['A3'].offset(0,col).value)
              
        stripped_col_heads = []
        for head in sheet_col_heads:
            stripped_col_heads.append(head.strip())
            
        stripped_col_heads2 = []
        for head in sheet_col_heads2:
            stripped_col_heads2.append(head.strip())

        compare = list(df_dict["df"][idx].columns)
        compare2 = []
        
        for header in compare:
            compare2.append(header.replace("\r","").replace("\n","").strip())
            
        print(f"{worksheet}: {stripped_col_heads == compare2}")
        
        
        if stripped_col_heads == compare2:
            pass
        else:
            print(f"sheet column headers don't agree with scraped scraped column headers")
            break
            
            
        #Insert Values from dataframe into excel with some format
        row_height = sheet_ranges.row_dimensions[last_row_previous].height
        for col in range(1,len(stripped_col_heads) + 1):
            for rw in range(ins_begin, ins_end + 1):
                #pasting value in cell
                try:
                    sheet_ranges.cell(rw,col).value = float((df_dict["df"][idx][stripped_col_heads[col - 1]][rw-ins_begin]).replace(",",""))
                    sheet_ranges.cell(rw,col).number_format = '#,###.0'
                except ValueError:
                    sheet_ranges.cell(rw,col).value = (df_dict["df"][idx][stripped_col_heads[col - 1]][rw-ins_begin]).replace(",","")
                #setting font style and size    
                sheet_ranges.cell(rw,col).font = Font(name='Arial', size=9)
                #setting border of cell
                border = Border(left=Side(border_style='thin', color='E4E4E4'),
                                right=Side(border_style='thin', color='E4E4E4'),
                                top=Side(border_style='thin', color='E4E4E4'),
                                bottom=Side(border_style='thin', color='E4E4E4'))
                sheet_ranges.cell(rw,col).border = border
                #setting alignment of text in cell
                alignment=Alignment(horizontal='right',
                                    vertical='top')
                sheet_ranges.cell(rw,col).alignment = alignment
                #setting row height
                sheet_ranges.row_dimensions[rw].height = row_height
                #setting fill color of cell
                greyFill = PatternFill(start_color='DCDCDC',
                           end_color='DCDCDC',
                           fill_type='solid')
                whiteFill = PatternFill(start_color='FFFFFF',
                    end_color='FFFFFF',
                    fill_type='solid')
                if rw % 2 == 0:
                    sheet_ranges.cell(rw,col).fill = greyFill 
                else:
                    sheet_ranges.cell(rw,col).fill = whiteFill
            print("column inserted successfully")
        
        idx += 1
    wb.save(filename = 'N:/Lsolis/eco/SHCP/Historicos/test.xlsm')

In [53]:
def add_to_used(used_already,xboundlists):
    for lst in xboundlists:
        el_id = lst._element.get_attribute('id')
        if el_id not in used_already:
            used_already.append(el_id)
    return used_already

In [54]:
def count_columns(sheet_ranges):
    non_empty = True
    col_ = 1
    col_count = 0
    while non_empty:
        if sheet_ranges['A3'].offset(0,col_ - 1).value:
            col_ +=1
            col_count += 1
        else:
            non_empty = False
    return col_count

In [72]:
app_dict = {
    "title":[
        "Balance del Sector Púbilco",
        "Situación Financiera del Sector Público",
        "Ahorro Corriente del Sector Público Presupuestario",
        "Financiamiento de la Inversión Física y Financiera Presupuestaria",
        "Financiamiento del Balance Público por Institución",
        "Financiamiento del Balance Público por Sector",
        "Ingresos Presupuestarios del Sector Público",
        "Ingresos Petroleros del Sector Público",
        "Ingresos Presupuestarios del Gobierno Federal, Petroleros y No petroleros",
        "Ingresos Presupuestarios del Gobierno Federal (Resumen Artículo 1 LIF)",
        "Ingresos Propios de Organismos y Empresas Bajo Control Presupuestario Directo",
        "Gasto Neto del Sector Público Presupuestario",
        "Gasto Programable del Sector Público Presupuestario",
        "Gasto Programable del Sector Público Presupuestario. Clasificación Funcional",
        "Gasto Programable del Sector Público Presupuestario. Clasidicación Funcional-Económica",
        "Servicios Personales del Sector Público Presupuestario. Clasfificación Funcional",
        "Otros Gastos de Operación del Sector Público Presupuestario. Clasificación Funcional",
        "Subsidios, Transferencias y Aportaciones del Sector Público Presupuestario distintos de Servicios Personales. Clasificación Funcional",
        "Ayudas y Otris Gastos Corrientes del Sector Oúblico Presupuestario. Clasificación Funcional",
        "Invesrsión Física del Sector Público Presupuestario. Clasificación Funcional",
        "Subsidios, Transferencias y Aportaciones del Sector Público Presupuestario Incluyendo Servicios Personales. Clasificación Funcional",
        "Inversión física presupuestaria",
        "Costo Financiero del Sector Público Presupuestario",
        "Costo Financiero del Sector Público Presupuestario. Clasificación Institucional",
        "Gasto del Gobierno Federal en Clasificación Económica",
        "Principales Beneficiarios de Transferencias del Gobierno Federal. Clasificación Funcional",
        "Participaciones a Entidades Federativas y Municipios por Fondo",
        "Adeudos de Ejercicios Fiscales Anteriores (ADEFAS). Clasificación Económica",
        "Adeudos de Ejercicios Fiscales Anteriores (ADEFAS). Clasificación Administrativa"
    ],
    "url":[
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Balance%20del%20Sector%20P%C3%BAblico&param_formato=1&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Situaci%C3%B3n%20Financiera%20del%20Sector%20P%C3%BAblico&param_formato=2&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ahorro%20Corriente%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=18&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Financiamiento%20de%20la%20Inversi%C3%B3n%20F%C3%ADsica%20y%20Financiera%20Presupuestaria&param_formato=19&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Financiamiento%20del%20Balance%20P%C3%BAblico%20por%20Instituci%C3%B3n&param_formato=20&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Financiamiento%20del%20Balance%20P%C3%BAblico%20por%20Sector&param_formato=21&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Presupuestarios%20del%20Sector%20P%C3%BAblico&param_formato=3&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Petroleros%20del%20Sector%20P%C3%BAblico&param_formato=29&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Presupuestarios%20del%20Gobierno%20Federal%2C%20Petroleros%20y%20No%20petroleros&param_formato=2&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Presupuestarios%20del%20Gobierno%20Federal%20%20(Resumen%20Art%C3%ADculo%201%20LIF)&param_formato=3&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ingresos%20Propios%20de%20Organismos%20y%20Empresas%20Bajo%20Control%20Presupuestario%20Directo&param_formato=4&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=3&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Neto%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=5&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20programable%20del%20sector%20p%C3%BAblico%20presupuestario&param_formato=15&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Programable%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=8&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Programable%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional-Econ%C3%B3mica&param_formato=9&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Servicios%20Personales%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=23&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Otros%20Gastos%20de%20Operaci%C3%B3n%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=24&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Subsidios%2C%20Transferencias%20y%20Aportaciones%20del%20Sector%20P%C3%BAblico%20Presupuestario%20distintos%20de%20Servicios%20Personales.%20Clasificaci%C3%B3n%20Funcional&param_formato=25&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ayudas%20y%20Otros%20Gastos%20Corrientes%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=26&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Inversi%C3%B3n%20F%C3%ADsica%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=27&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Subsidios%2C%20Transferencias%20y%20Aportaciones%20del%20Sector%20P%C3%BAblico%20Presupuestario%20Incluyendo%20Servicios%20Personales.%20Clasificaci%C3%B3n%20Funcional&param_formato=28&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Inversi%C3%B3n%20f%C3%ADsica%20presupuestaria&param_formato=11&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Costo%20Financiero%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=12&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Costo%20Financiero%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Institucional&param_formato=13&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20del%20Gobierno%20Federal%20en%20Clasificaci%C3%B3n%20Econ%C3%B3mica&param_formato=29&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Principales%20Beneficiarios%20de%20Transferencias%20del%20Gobierno%20Federal.%20Clasificaci%C3%B3n%20Funcional&param_formato=10&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Participaciones%20a%20Entidades%20Federativas%20y%20Municipios%20por%20Fondo&param_formato=17&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Adeudos%20de%20Ejercicios%20Fiscales%20Anteriores%20(ADEFAS).%20Clasificaci%C3%B3n%20Econ%C3%B3mica&param_formato=14&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Adeudos%20de%20Ejercicios%20Fiscales%20Anteriores%20(ADEFAS).%20Clasificaci%C3%B3n%20Administrativa&param_formato=15&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&"
    ],
    "sheet":[
        "1.1.1S",
        "1.1.2S",
        "1.1.3S",
        "1.1.4S",
        "1.1.5S",
        "1.1.6S",
        "1.2.1S",
        "1.2.2S",
        "1.2.3S",
        "1.2.4S",
        "1.2.5S",
        "1.3.1S",
        "1.3.2S",
        "1.3.3S",
        "1.3.4S",
        "1.3.5S",
        "1.3.6S",
        "1.3.7S",
        "1.3.8S",
        "1.3.9S",
        "1.3.10S",
        "1.3.11S",
        "1.3.12S",
        "1.3.13S",
        "1.3.14S",
        "1.3.15S",
        "1.3.16S",
        "1.3.17S",
        "1.3.18S"
    ]
}

In [64]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

base_url = "http://presto.hacienda.gob.mx"
#table_url = "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Balance%20del%20Sector%20P%C3%BAblico&param_formato=1&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1& "

#url = "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Balance%20del%20Sector%20P%C3%BAblico&param_formato=1&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&"
#url = "http://www.banxico.org.mx/"


In [65]:
df_dict = {
    "title":[],
    "df":[],
    "sheet":[]
}
for i in range(0,len(app_dict["title"])):
    df = scrape(i,browser,app_dict)
    title = browser.find_by_css(".cls-title-table").find_by_css(".cls-title-main").text
    df_dict["title"].append(title)
    df_dict["df"].append(df)
    df_dict["sheet"].append(app_dict["sheet"][i])

looking for element. attempt number 1
looking for element. attempt number 1
walao
walao
walao
walao
consultar button clicked
alert accepted
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
no consultar button
consultar button clicked
there is no alert
test = [<splinter.driver.webdriver.WebDriverElement object at 0x0E9CAF90>, <splinter.driver.webdriver.WebDriverElement object at 0x0CADCCF0>]
loading...
walao
walao
walao
walao
walao
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found yet...
not found ye

test = [<splinter.driver.webdriver.WebDriverElement object at 0x0EB07F90>, <splinter.driver.webdriver.WebDriverElement object at 0x0EB07670>]
loading...


In [79]:
store(df_dict)

1.3.1S: True
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
1.3.2S: True
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted s

column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
1.3.11S: True
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
column inserted successfully
1.3.12S: True
column inserted successfully
column inserted successfully
column inserted successfully
column inserted

In [73]:
app_dict = {
    "title":[
        "Gasto Neto del Sector Público Presupuestario",
        "Gasto Programable del Sector Público Presupuestario",
        "Gasto Programable del Sector Público Presupuestario. Clasificación Funcional",
        "Gasto Programable del Sector Público Presupuestario. Clasidicación Funcional-Económica",
        "Servicios Personales del Sector Público Presupuestario. Clasfificación Funcional",
        "Otros Gastos de Operación del Sector Público Presupuestario. Clasificación Funcional",
        "Subsidios, Transferencias y Aportaciones del Sector Público Presupuestario distintos de Servicios Personales. Clasificación Funcional",
        "Ayudas y Otris Gastos Corrientes del Sector Oúblico Presupuestario. Clasificación Funcional",
        "Invesrsión Física del Sector Público Presupuestario. Clasificación Funcional",
        "Subsidios, Transferencias y Aportaciones del Sector Público Presupuestario Incluyendo Servicios Personales. Clasificación Funcional",
        "Inversión física presupuestaria",
        "Costo Financiero del Sector Público Presupuestario",
        "Costo Financiero del Sector Público Presupuestario. Clasificación Institucional",
        "Gasto del Gobierno Federal en Clasificación Económica",
        "Principales Beneficiarios de Transferencias del Gobierno Federal. Clasificación Funcional",
        "Participaciones a Entidades Federativas y Municipios por Fondo",
        "Adeudos de Ejercicios Fiscales Anteriores (ADEFAS). Clasificación Económica",
        "Adeudos de Ejercicios Fiscales Anteriores (ADEFAS). Clasificación Administrativa"
    ],
    "url":[
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Neto%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=5&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20programable%20del%20sector%20p%C3%BAblico%20presupuestario&param_formato=15&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Programable%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=8&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20Programable%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional-Econ%C3%B3mica&param_formato=9&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Servicios%20Personales%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=23&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Otros%20Gastos%20de%20Operaci%C3%B3n%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=24&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Subsidios%2C%20Transferencias%20y%20Aportaciones%20del%20Sector%20P%C3%BAblico%20Presupuestario%20distintos%20de%20Servicios%20Personales.%20Clasificaci%C3%B3n%20Funcional&param_formato=25&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Ayudas%20y%20Otros%20Gastos%20Corrientes%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=26&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Inversi%C3%B3n%20F%C3%ADsica%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Funcional&param_formato=27&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Subsidios%2C%20Transferencias%20y%20Aportaciones%20del%20Sector%20P%C3%BAblico%20Presupuestario%20Incluyendo%20Servicios%20Personales.%20Clasificaci%C3%B3n%20Funcional&param_formato=28&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Inversi%C3%B3n%20f%C3%ADsica%20presupuestaria&param_formato=11&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Costo%20Financiero%20del%20Sector%20P%C3%BAblico%20Presupuestario&param_formato=12&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Costo%20Financiero%20del%20Sector%20P%C3%BAblico%20Presupuestario.%20Clasificaci%C3%B3n%20Institucional&param_formato=13&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=1&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Gasto%20del%20Gobierno%20Federal%20en%20Clasificaci%C3%B3n%20Econ%C3%B3mica&param_formato=29&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Principales%20Beneficiarios%20de%20Transferencias%20del%20Gobierno%20Federal.%20Clasificaci%C3%B3n%20Funcional&param_formato=10&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Participaciones%20a%20Entidades%20Federativas%20y%20Municipios%20por%20Fondo&param_formato=17&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Adeudos%20de%20Ejercicios%20Fiscales%20Anteriores%20(ADEFAS).%20Clasificaci%C3%B3n%20Econ%C3%B3mica&param_formato=14&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&",
        "/presto/files/system/mashlets/app_layout_estopor/index.html?param_formato_desc=Adeudos%20de%20Ejercicios%20Fiscales%20Anteriores%20(ADEFAS).%20Clasificaci%C3%B3n%20Administrativa&param_formato=15&param_unidad=1&param_tipo=1&param_lenguaje=1&param_clasificacion=2&"
    ],
    "sheet":[
        "1.3.1S",
        "1.3.2S",
        "1.3.3S",
        "1.3.4S",
        "1.3.5S",
        "1.3.6S",
        "1.3.7S",
        "1.3.8S",
        "1.3.9S",
        "1.3.10S",
        "1.3.11S",
        "1.3.12S",
        "1.3.13S",
        "1.3.14S",
        "1.3.15S",
        "1.3.16S",
        "1.3.17S",
        "1.3.18S"
    ]
}

In [74]:
for key in list(app_dict.keys()):
    print(len(app_dict[key]))

18
18
18


In [77]:
df_dict["df"][1]

,Fecha(Mensual),XAC10 - Gasto programable presupuestario pagado,XAC40 - Gasto programable del Gobierno Federal,XDB50 - Ramos autónomos,XDB54 - Legislativo,XDB55 - Judicial,XDB56 - Instituto Federal Electoral,XDB57 - Comisión Nacional de los Derechos Humanos,XDB40 - Instituto Nacional de Estadística y Geografía,XAC4380 - Gasto programable del Ramo Tribunal Fiscal de la Federación,...,XAC4390 - Gasto programable del Ramo Aportaciones Federales para Entidades Federativas y Municipios,XAC50 - Gasto programable de entidades bajo control presupuestario directo,XAC51 - Gasto programable de Pemex,XAC52 - Gasto programable de organismos y empresas diferentes de Pemex,XOA0141 - CFE,XOA0142 - LFC,XOA0145 - IMSS,XOA0146 - ISSSTE,XOA0112 - Lotenal,XAC5210 - Operaciones compensadas
0,01/2019,"366,742.8","264,001.6","5,426.6",656.7,"2,948.2",817.6,72.7,630.1,138.6,...,"65,412.8","205,784.6","49,948.6","155,836.1","64,346.4",0.0,"45,261.9","46,227.7",0.0,"103,043.4"
1,02/2019,"260,959.7","189,789.9","6,418.3",753.5,"3,314.7","1,064.2",89.7,831.4,149.2,...,"57,957.3","131,976.3","28,969.1","103,007.2","29,569.7",0.0,"46,968.3","26,469.2",0.0,"60,806.5"
2,03/2019,"335,405.3","249,338.2","7,083.5",785.5,"3,696.6","1,222.4",146.8,867.4,142.8,...,"55,543.5","145,592.3","23,632.4","121,959.9","38,039.3",0.0,"55,338.3","28,582.2",0.0,"59,525.1"
3,04/2019,"304,781.4","221,362.8","9,455.1",910.1,"5,963.6","1,166.2",111.9,895.2,165.6,...,"55,164.2","144,192.0","29,860.9","114,331.1","36,050.7",0.0,"52,097.7","26,182.6",0.0,"60,773.4"
4,05/2019,"324,616.7","237,563.3","7,413.4",903.7,"3,875.7","1,294.8",122.6,822.0,161.9,...,"63,958.1","146,439.8","31,345.2","115,094.6","34,522.2",0.0,"57,047.0","23,525.5",0.0,"59,386.4"
5,06/2019,"331,541.4","254,804.9","7,753.1",885.8,"4,146.6","1,499.2",122.3,691.5,152.7,...,"58,913.0","142,002.2","28,844.4","113,157.8","30,041.3",0.0,"55,527.6","27,588.8",0.0,"65,265.7"


In [70]:
sheet_ranges = wb["1.3.2S"]

NameError: name 'wb' is not defined

In [175]:
idx = 0
for worksheet in df_dict["sheet"]:
    #Making sure headers in excel are equal to headers in dataframe
    sheet_ranges = wb[worksheet]
    sheet_col_heads = []
    
    for col in range(0,count_columns(sheet_ranges)):
        sheet_col_heads.append(sheet_ranges['A3'].offset(0,col).value.replace("_x000D_",""))

    stripped_col_heads = []
    for head in sheet_col_heads:
        stripped_col_heads.append(head.strip())

    compare = list(df_dict["df"][idx].columns)
    compare2 = []
    for header in compare:
        compare2.append(header.replace("\r","").replace("\n","").strip())
    print(f"{worksheet}: {stripped_col_heads == compare2}")
    idx += 1

1.1.1S: True
1.1.2S: True
1.1.3S: True
1.1.4S: True
1.1.5S: True
1.1.6S: True
1.2.1S: True
1.2.2S: True
1.2.3S: True
1.2.4S: True
1.2.5S: True


In [169]:
#Making sure headers in excel are equal to headers in dataframe
sheet_ranges = wb[df_dict["sheet"][8]]
sheet_col_heads = []
    
for col in range(0,count_columns(sheet_ranges)):
    sheet_col_heads.append(sheet_ranges['A3'].offset(0,col).value.replace("_x000D_",""))

stripped_col_heads = []
for head in sheet_col_heads:
    stripped_col_heads.append(head.strip())

compare = list(df_dict["df"][8].columns)
compare2 = []
for header in compare:
    compare2.append(header.replace("\r","").replace("\n",""))
print(f"{stripped_col_heads == compare2}")

False


In [170]:
for i in range(0,len(stripped_col_heads)):
    if compare2[i] == stripped_col_heads[i]:
        print(f"TRUE: {compare2[i]} = {stripped_col_heads[i]}")
        print(" ")
    else:
        print(f"FALSE: {compare2[i]} != {stripped_col_heads[i]}")
        print(" ")

TRUE: Fecha(Mensual) = Fecha(Mensual)
 
TRUE: XNA0101 - Ingresos presupuestarios = XNA0101 - Ingresos presupuestarios
 
TRUE: XNA0102 - Petroleros = XNA0102 - Petroleros
 
TRUE: XNA0227 - Transferencias del fondo mexicano del petróleo para la estabilización y el desarrollo = XNA0227 - Transferencias del fondo mexicano del petróleo para la estabilización y el desarrollo
 
TRUE: XNA0246 - ISR de contratistas y asignatarios = XNA0246 - ISR de contratistas y asignatarios
 
TRUE: XNA0206 - Hidrocarburos = XNA0206 - Hidrocarburos
 
TRUE: XNA0115 - Aprovechamientos s/rendimientos excedentes. = XNA0115 - Aprovechamientos s/rendimientos excedentes.
 
TRUE: XNA0118 - No petroleros = XNA0118 - No petroleros
 
TRUE: XNA0200 - Ingresos tributarios = XNA0200 - Ingresos tributarios
 
TRUE: XNA0220 - ISR-IETU-IDE = XNA0220 - ISR-IETU-IDE
 
TRUE: XNA0120 - Impuesto Sobre la Renta = XNA0120 - Impuesto Sobre la Renta
 
TRUE: XNA0209 - Impuesto empresarial a tasa unica = XNA0209 - Impuesto empresarial a t